In [3]:
#подключение диска
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# импорт всего и вся
import matplotlib
import pandas as pd
matplotlib.use("Agg")

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.layers import Conv2D, Flatten, Dropout, Activation, MaxPooling2D
from keras.optimizers import Adam

from tensorflow.keras.utils import plot_model
from IPython.display import display, Image

from keras.models import load_model
from imutils import  paths

import matplotlib
import pandas as pd
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle

import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [62]:
data=pd.read_csv("/content/drive/My Drive/Банкоматы/nen.csv")
data

,id,city-address,type,passability,area,Average of money,Average of profit,Average of our_clients,Count of Index
0,1,"г. Хабаровск, ул. Суворова, д. 25",0,61744,7899.6,17071351.75,0.005,0.140000,8.0
1,2,"г. Владивосток, ул. Черемуховая, д. 15",6,30364,2529.2,2194189.29,0.005,0.135714,7.0
2,3,"г. Хабаровск, ул. Карла Маркса, д. 91",0,7220,38.1,9431.43,0.005,0.148571,7.0
3,4,"г. Хабаровск, ул. Суворова, д. 51",0,94443,4008.8,9591278.50,0.003,0.040000,6.0
4,5,"г. Владивосток, пр-кт. 100-летия Владивостока,...",0,45105,1394.4,2616407.60,0.007,0.198000,5.0
...,...,...,...,...,...,...,...,...,...
8114,8115,"г. Хабаровск, ш. Матвеевское, д. 6",0,49203,4168.3,NaN,NaN,NaN,NaN
8115,8116,"г. Хабаровск, ш. Матвеевское, д. 7",0,50133,2713.3,NaN,NaN,NaN,NaN
8116,8117,"г. Хабаровск, ш. Матвеевское, д. 8",0,69810,4196.8,NaN,NaN,NaN,NaN
8117,8118,"г. Хабаровск, ш. Матвеевское, д. 9",6,58225,3021.5,NaN,NaN,NaN,NaN


In [64]:
pred[7]

array([672180.75], dtype=float32)

In [57]:
# загрузка Data сета и подготовка данных

data=pd.read_csv("/content/drive/My Drive/Банкоматы/nen.csv") # загрузка данных о 
model = load_model("/content/drive/My Drive/Банкоматы/EasyNetF.model")

datas=[]
Profit=[]
Our_clients=[]
for i in range(data.shape[0]):
  z = data.loc[i].values
  if False == np.isnan(z[6]):
    Profit.append((z[6]))
  if False == np.isnan(z[7]):
    Our_clients.append(z[7])
  z = z[2:5]
  datas.append(z)
control_list=[]
data=datas
Profit = (np.array(Profit,dtype="float"))
Our_clients= (np.array(Our_clients,dtype="float"))
pred = model.predict(np.array(data,dtype="float"))
Profit=Profit.mean() # нахождение средней процентонй прибыли от наших клиентов на наших банкоматах
Our_clients=Our_clients.mean() # нахождени средней процентоной прибыли от наших клиентов на не наших банкоматах и 
# не наших клиентов на аших банкоматах

In [115]:
GAIN=[]
quantity=[]

asd=[]
for i in range(len(pred)):
  asd.append(pred[i][0])

# цикл в котором мы перебираем все возможные расстановки банкоматов прогоняя каждую перестановку через алгоритм рассчета прибыли
for i in range(len(data)):
  gain=[]
  for x in range(5):

    banks = x


    banks2 = 0

    if banks!=0:
      banks=(1/banks)
      banks2=0
    else:
      banks2=1 # отсутствие здесь нашего бакомата 

      #ограничение оборота на одном банкомате
    dv=pred[i][0]

    #ограничение оборота на банкоматах
    if x != 0:
      if dv/x>20000000:
        dv-=(pred[i][0]/x-20000000)

    # доход с наших банкоматов от наших клиентов
    gain1= dv*banks*(Profit-0.0015) 

    # доход с наших банкоматов от ненаших клиентов 
    gain2= pred[i][0]*(Our_clients/100)*banks 

    # доход c наших клиентов не на наших банкоматах
    gain3= pred[i][0]*(Our_clients/100)*banks2

    # Общий доход
    gainX=gain1*x+gain2+gain3 -x*5000 - 0.0015*pred[i][0]*x
    # где x*5000 расход на аренду площади. 0.0015*pred[i][0]*x расход на инкасацию 

    gain.append(gainX)
  
  quantity.append(gain.index(max(gain)))
  GAIN.append(max(gain))

In [101]:
# отладка модели алгоритма

gain=[-10000]
for x in range(5):
    i=7
    banks = x


    banks2 = 0

    if banks!=0:
      banks=(1/banks)
      banks2=0
    else:
      banks2=1 # отсутствие здесь нашего бакомата 

      #ограничение оборота на одном банкомате
    dv=pred[i][0]
    if x != 0:
      if dv/x>10000000:
        dv-=(pred[i][0]/x-10000000)

    # доход с наших банкоматов от наших клиентов
    gain1= dv*banks*(Profit-0.0015) 

    # доход с наших банкоматов от ненаших клиентов 
    gain2= pred[i][0]*(Our_clients/100)*banks 

    # доход c наших клиентов не на наших банкоматах
    gain3= pred[i][0]*Our_clients/100*banks2

    # Общий доход
    gainX=gain1*x+gain2+gain3 -x*20000 - 0.0015*pred[i][0]*x
    print(gain)
    gain.append(gainX)

[-10000]
[-10000, 501.26825881617924]
[-10000, 501.26825881617924, -18618.816739299396]
[-10000, 501.26825881617924, -18618.816739299396, -39877.721993707484]
[-10000, 501.26825881617924, -18618.816739299396, -39877.721993707484, -60969.53782851018]


In [125]:
#Сохранение csv файла с предсказаным оборотом, кол-вом банкомаов в каждой точке и выручкой

data=pd.read_csv("/content/drive/My Drive/Банкоматы/nen.csv")
FINAL  = pd.DataFrame({'Average of money new': asd,
                      'quantity': quantity,

                      'GAIN': GAIN},

                      index = [i for i in range(8119)])
del data['Average of money']
data = data.join(FINAL)
data.to_csv("/content/drive/My Drive/Банкоматы/AnswerF.csv")